# Imports and settings

In [154]:
from openmm.app import *
from openmm import *
from openmm.unit import *

# Create system + CV Forces with harmonic restraints

In [155]:
# Load an already solvated PDB file and set up the system + state
pdb = PDBFile("../villin.pdb")
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology,
                                         nonbondedMethod=PME,
                                         nonbondedCutoff=10.0 * angstrom,
                                         constraints=HBonds,
                                         rigidWater=True,
                                         hydrogenMass=4.0 * amu)


In [156]:
# Define three atom indices - these will be used to define distance CVs
d1_atom1_ind = 83
d1_atom2_ind = 151
d2_atom2_ind = 254

In [157]:
# Create Harmonic Force that operate on the CVs 
bias = HarmonicBondForce()
bias.addBond(d1_atom1_ind,
             d1_atom2_ind,
             0.3,
             1.0)

bias.addBond(d1_atom1_ind,
             d2_atom2_ind,
             0.3,
             1.0)

1

In [158]:
# Add force to the system
system.addForce(bias)

5

Ultimately we want to track the value of these distances, which we will use to create our free energy projection – let's call them  D1 and D2.\
To track D1 and D2, we could use a whole `Reporter` object to save the whole trajectory and measure it afterwards.\
A lightweight way of doing this without the whole trajectory is to use a `customCVForce` object set to 0 (i.e. no bias). This will compute the value of a CV at each step of the simulation.\
We then pass D1 and D2 as bondForces to the `customCVForce` object (using `r` to track their distance)

In [159]:
# define a distance measurer
dist_measurer = CustomCVForce("0")
# Create our two distances as separate Bond Forces and add them
# Note: If we added both D1 and D2 to the same BondForce, then doing 
#     dist_measurer.getCollectiveVariableValues will return only the first distance (D1)
# Hence we define them as separate BondForces
D1 = CustomBondForce("r")
D1.addBond(d1_atom2_ind, d1_atom1_ind)
D2 = CustomBondForce("r")
D2.addBond(d2_atom2_ind, d1_atom1_ind)

# Add each BondForce as CVs into the dist_measurer
dist_measurer.addCollectiveVariable("D1", D1)
dist_measurer.addCollectiveVariable("D2", D2)
system.addForce(dist_measurer)

6

# define Integrator and create the Simulation object

In [160]:
integrator = LangevinMiddleIntegrator(300*kelvin,
                                      1.0/picosecond,
                                      0.002 * picosecond)

In [161]:
simulation = Simulation(pdb.topology,system, integrator)

# Set up a single simulation with one umbrella (in reality you'd do multiple of these)


In [162]:
simulation.context.setPositions(pdb.positions)

In our simulation, to generate an Free Energy Profile (or a Potential of Mean Force - PMF) we need 4 things:
1. A set of features that each biasing potential will be projected upon (D1 and D2)
2. A biasing potential that upon an specific CV value for each window (our harmonic constants)
3. The actual measured D1 and D2 values for each frame, for each window 
4. For each frame, the calculated total Potential energy

Using our `dist_measurer`, we can track D1 and D2 for each frame of the simulation.\
For both D1 and D2, we also know the value of the biasing potential that was applied (using the constants `k` and `r0`). From this we can compute the Reduced Free Energy at each frame per window.\
By computing the Reduced Free Energy for each frame across multiple windows, we can then use 
thermodynamic estimators like MBAR to compute the PMF.
For more details on this analysis, check out the PyMBAR documentation: https://pymbar.readthedocs.io/en/master/fes_with_pymbar.html

In [164]:
for i in range(5):
    simulation.step(10)
    n_steps = str(simulation.context.getStepCount())
    v0 = simulation.context.getState(getEnergy=True).getPotentialEnergy()
    d1,d2 = dist_measurer.getCollectiveVariableValues(simulation.context)
    print(n_steps, v0, d1, d2)

60 -127028.98186972504 kJ/mol 0.44531336426734924 0.4284404516220093
70 -127002.45061972504 kJ/mol 0.45564672350883484 0.4295880198478699
80 -127204.38811972504 kJ/mol 0.46104344725608826 0.4328082203865051
90 -127126.54436972504 kJ/mol 0.47314080595970154 0.4301340579986572
100 -126815.85686972504 kJ/mol 0.4680291414260864 0.41703155636787415


Note that the above simulation is done for a _single_ window. To compute the PMF, we need to run multiple simulations, each with a different biasing potential applied (ie a different window).\ 

The easiest way to set up multiple windows, you'll just want to set up multiple `system` objects.\
For each `system`, you'll define a set of umbrellas with something like:

```
M = 20
r0_range = np.linspace(0.3, 2.0, M, endpoint = False)
```

Then for each new system (1 per window), you'd can define it after `context` construction: 
```
simulation.context.setParameter('r0_d1', r0_range[m])
```